In [1]:
import os
from src.util import eval_util as eutil


REPO_DIR = os.path.join(os.getcwd(), "../")
os.chdir(REPO_DIR)

CUDA GPU: False


In [2]:
tracerx_calibrate='/data/morrisq/divyak/projects/met_history_prediction/src/data/tracerx_nsclc/metient_outputs/pyclone_conipher_calibrate__nositenumrestriction_r2__2023/calibrate/'
eutil.get_max_cross_ent_thetas(pickle_file_dirs=[tracerx_calibrate])

Restricting to patients wih > 2 sites
num patients 128 50
Step 0: theta = tensor([0.3333, 0.3333, 0.3333], grad_fn=<DivBackward0>), Objective = 87.63581848144531
Step 10: theta = tensor([0.3443, 0.3300, 0.3256], grad_fn=<DivBackward0>), Objective = 87.61882781982422
Step 20: theta = tensor([0.3547, 0.3268, 0.3184], grad_fn=<DivBackward0>), Objective = 87.60414123535156
Step 30: theta = tensor([0.3645, 0.3238, 0.3117], grad_fn=<DivBackward0>), Objective = 87.59149169921875
Step 40: theta = tensor([0.3735, 0.3210, 0.3055], grad_fn=<DivBackward0>), Objective = 87.5807113647461
Step 50: theta = tensor([0.3819, 0.3183, 0.2997], grad_fn=<DivBackward0>), Objective = 87.5715560913086
Step 60: theta = tensor([0.3897, 0.3159, 0.2944], grad_fn=<DivBackward0>), Objective = 87.56381225585938
Step 70: theta = tensor([0.3969, 0.3137, 0.2895], grad_fn=<DivBackward0>), Objective = 87.55728912353516
Step 80: theta = tensor([0.4034, 0.3117, 0.2849], grad_fn=<DivBackward0>), Objective = 87.55184173583984


[0.4478316605091095, 0.30158117413520813, 0.2505871653556824]

In [ ]:
prediction_dir = os.path.join(REPO_DIR, 'test', 'machina_simulated_data', 'predictions_bs1024_calibrate_solvepoly_wip_thetaopt_12072023')
eutil.get_max_cross_ent_thetas(pickle_file_dirs=[os.path.join(prediction_dir, 'm8', 'mS'),os.path.join(prediction_dir, 'm5', 'mS')])
eutil.get_max_cross_ent_thetas(pickle_file_dirs=[os.path.join(prediction_dir, 'm8', 'R'),os.path.join(prediction_dir, 'm5', 'R')])


In [ ]:

def is_ground_truth_tree_parallel_single_source(site, mig_type, seed):
    labeling_fn = os.path.join(SIMS_DIR, site, mig_type, f"T_seed{seed}.vertex.labeling")
    tree_fn = os.path.join(SIMS_DIR, site, mig_type, f"T_seed{seed}.tree")
    true_edges, true_mig_edges, true_labeling = eutil.parse_clone_tree(tree_fn, labeling_fn)
    del true_labeling['GL']
    num_nodes = len(true_labeling)
    num_sites = len(set(list(true_labeling.values())))
    node_label_to_idx = {k:i for i,k in enumerate(list(true_labeling.keys()))}
    A = torch.zeros((num_nodes, num_nodes))
    for edge in true_edges:
        if edge[0] == "GL":
            continue
        A[node_label_to_idx[edge[0]], node_label_to_idx[edge[1]]] = 1
    
    site_label_to_idx = {k:i for i,k in enumerate(set(list(true_labeling.values())))}
    V = torch.zeros((num_sites, num_nodes))
    for node_label in true_labeling:
        site_label = true_labeling[node_label]
        V[site_label_to_idx[site_label], node_label_to_idx[node_label]] = 1
    
    m, c, s, _, _ = get_ancestral_labeling_metrics(add_batch_dim(V), A, None, None, None)    
    return s.item()==1.0

parallel_ss_trees = []
nonparallel_ss_trees = []

for mig_type in MIG_TYPES:
    for site in SITES:
        site_mig_type_dir = os.path.join(SIMS_DIR, site, mig_type)
        seeds = fnmatch.filter(os.listdir(site_mig_type_dir), 'clustering_observed_seed*.txt')
        seeds = [s.replace(".txt", "").replace("clustering_observed_seed", "") for s in seeds]
        for seed in seeds:
            is_parallel_ss = is_ground_truth_tree_parallel_single_source(site, mig_type, seed)
            num_trees = get_num_mut_trees(os.path.join(SIMS_DIR, f"{site}_mut_trees", f"mut_trees_{mig_type}_seed{seed}.txt"))
            for tree_num in range(num_trees):
                seeding_fn = os.path.join(prediction_dir, site, mig_type, f"tree{tree_num}_seed{seed}_calibrate.pkl.gz")
                if is_parallel_ss:
                    parallel_ss_trees.append(seeding_fn) 
                else:
                    nonparallel_ss_trees.append(seeding_fn)
print(len(parallel_ss_trees),len(nonparallel_ss_trees)) 

eutil.get_max_cross_ent_thetas(pickle_file_list=parallel_ss_trees)
eutil.get_max_cross_ent_thetas(pickle_file_list=nonparallel_ss_trees)
